## 读取比赛所有的pdf并向量化后存入es数据库

In [ ]:
import glob2
files_path = glob2.glob('/Users/yangjie/Documents/python_project/zhongxing/参考文献/*')
print(files_path)

In [ ]:
import os 
import json
from openai import OpenAI
from pathlib import Path
def convert_pdf2txt(file_path):
    client_f = OpenAI(
        api_key="sk-rM8bGJ3AOHI5hR9NhL3SVqi31aDJfIIs36Xbl55RENZekM9S",
        base_url="https://api.moonshot.cn/v1",
    )
    file_object = client_f.files.create(
        file=Path(file_path), purpose="file-extract"
    )
    file_content = json.loads(
        client_f.files.content(file_id=file_object.id).text
    )
    fnw = os.path.splitext(file_path)[0] + ".txt"
    with open(fnw, "w") as file:
        file.write(str(file_content["content"]))

for file_path in files_path:
    convert_pdf2txt(file_path)

## 切分文本

In [ ]:
from text_split import ChineseRecursiveTextSplitter
text_splitter = ChineseRecursiveTextSplitter(
    keep_separator=True, is_separator_regex=True, chunk_size=1000, chunk_overlap=50
)
def get_text_from_txt(file_path):
    input_text = ''
    with open(file_path,'r',encoding='utf-8') as f:
        lines = f.readlines()
        input_text += '\n'.join(lines)
    return input_text
files_path_cn = [
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/cn202303.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/cn202304.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/cn202305.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/cn202306.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/cn202401.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/cn202402.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/jx202311.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/jx202312.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/jx202401.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/jx202402.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/jx202403.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/jx202404.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/jx202405.txt',
]
files_path_en = [
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/en202303.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/en202304.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/en202401.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/ZTE TECHNOLOGIES (No.1) 2024.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/ZTE TECHNOLOGIES (No.2) 2024.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/ZTE TECHNOLOGIES (No.3) 2023.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/ZTE TECHNOLOGIES (No.3) 2024.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/ZTE TECHNOLOGIES (No.4) 2023.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/ZTE TECHNOLOGIES (No.5) 2023.txt',
    '/Users/yangjie/Documents/python_project/zhongxing/参考文献/ZTE TECHNOLOGIES (No.6) 2023.txt',
]
ls_cn,ls_en = [],[]
chunks_cn,chunks_en = [],[]
for file_path in files_path_cn:
    ls_cn.append(get_text_from_txt(file_path))
for inum, text in enumerate(ls_cn):
    chunks = text_splitter.split_text(text)
    for chunk in chunks:
        chunks_cn.append(chunk)
for file_path in files_path_en:
    ls_en.append(get_text_from_txt(file_path))
for inum, text in enumerate(ls_en):
    chunks = text_splitter.split_text(text)
    for chunk in chunks:
        chunks_en.append(chunk)

print(len(chunks_cn))
print(len(chunks_en))

## 向量化入库

In [ ]:
from es_kb_service import ESKBService
esKBService = ESKBService()
data_list = chunks_en
esKBService.do_add_doc(data_list)

## 根据语句检索数据库内容判断对错

In [ ]:
def get_questions():
    import csv
    csv_file_path = '/Users/yangjie/Documents/python_project/zhongxing/test_B.csv'
    questions = []
    with open(csv_file_path, mode='r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            questions.append(row['question'])
    return questions

In [ ]:
from langchain.chat_models import ChatOpenAI
def post_query(query):
    model = ChatOpenAI(
            openai_api_key="sk-b3RjMZgQjoCCQzqA2dD4F232532645Ab9948062611E45171",
            openai_api_base="http://10.120.18.242:30387/v1",
            model_name="qwen2-72b",
            temperature=0.2,
        )
    model = ChatOpenAI(
            openai_api_key="EMPTY",
            openai_api_base='http://10.13.14.16:7863/v1',
            model_name='glm-4-9b-chat',
            temperature=0.2, 
            model_kwargs={"stop": ["<|endoftext|>", "<|user|>", "<|observation|>"], "extra_body": {"skip_special_tokens": False}}  
        )
    answer = model.invoke(query).content
    return answer

# import os
# from concurrent.futures import ThreadPoolExecutor, as_completed
# from typing import List, Literal, Optional, Callable, Generator, Dict, Any, Awaitable, Union

# thread_pool = ThreadPoolExecutor(os.cpu_count())

# def run_in_thread_pool_v2(
#     func: Callable,
#     params: List[Dict] = [],
#     pool: ThreadPoolExecutor = None,
# ):
#     """
#     在线程池中批量运行任务，并将运行结果以生成器的形式返回。
#     请确保任务中的所有操作是线程安全的，任务函数请全部使用关键字参数。
#     """
#     tasks = []
#     pool = ThreadPoolExecutor(os.cpu_count())

#     for kwargs in params:
#         thread = pool.submit(func, **kwargs)
#         tasks.append(thread)
#     results = []
#     for obj in as_completed(tasks):
#         results.append(obj.result())
#     return results
print(post_query('你好'))

## 自一致性

In [ ]:
def get_answer(answer):
    answer = answer.split('。')[0]
    if '正确' in answer:
        res = 'T'
    elif '错误' in answer:
        res = 'F'
    else:
        res = '未知'
    return res
def get_self_consistence_answer(input_query,num):
    from collections import Counter
    answers = []
    for _ in range(num):
        answers += [get_answer(post_query(input_query))]
    counter = Counter(answers)
    most_common_element, _ = counter.most_common(1)[0]
    return most_common_element



## 检索相关文档chunks并进行重排序

In [ ]:
from es_kb_service import ESKBService
from FlagEmbedding import FlagReranker

query_list = get_questions()
res_list = []

def get_answer_from_LLM(query):
    print('query:',query)
    # 检索之前先对问题进行翻译
    input_query = f"""请将以下句子进行翻译，如果用户输入是中文就翻译成英文，如果用户输入是英文就翻译成中文。
【注意】直接输出翻译结果，不要输出其他多余的内容

用户输入：{query}
"""
    answer_tans = post_query(input_query)
    print(answer_tans)
    esKBService = ESKBService()
    docs = esKBService.do_search(query+answer_tans,top_k=50)
    # 对检索出来的docs进行重排序
    # reranker = FlagReranker("BAAI/bge-reranker-large", use_fp16=True)
    # scores = reranker.compute_score(
    #     [[query+answer_tans, doc.page_content] for doc in docs]
    # )
    # docs = [
    #     doc
    #     for doc, _ in sorted(
    #         zip(docs, scores), key=lambda pair: pair[1], reverse=True
    #     )
    # ][:50]
    # 调用大模型给出判断
    context = ''
    for inum, doc in enumerate(docs):
        doc = doc.replace('\n','')
        context += f"""知识 [{inum}]： {doc}\n"""
    input_query = f"""你是一个做判断题的能手，请结合给定的知识，判断用户输入的语句是否正确，如果参考文档中未提及则不算错误
【注意】回答格式请统一输出，如：
如果选项正确，则输出：正确。原因是***
如果选项错误，则输出：错误。原因是***
【注意】回答请精简准确，不超过50字，注意一定要谨慎小心可能错误的选项，并结合已有的常识综合进行判断。

<参考消息>
{context}
<参考消息>

用户输入：{query}
"""
    # print(input_query)
    answer = get_self_consistence_answer(input_query,5)
    # answer = post_query(input_query)
    print(answer)
    return answer

for i in range(0,100):
    print(f'第{i+1}题......')
    res = get_answer_from_LLM(query_list[i])
    res_list+=[res]

## 转换得到结果

In [ ]:
# 从答案列表中转换TF
TF_list = []
for answer in res_list:
    answer = answer.split('。')[0]
    if '正确' in answer:
        TF_list.append('T')
    elif '错误' in answer:
        TF_list.append('F')
    else:
        TF_list.append('未知')
print(TF_list)

In [ ]:
import csv

csv_file_path = '/Users/yangjie/Documents/python_project/zhongxing/result_v2.csv'

with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    row_to_write = ['id', 'answer']
    writer.writerow(row_to_write)
    for i,answer in enumerate(TF_list):
        row_to_write = [i+1, answer]
        writer.writerow(row_to_write)

print(f"数据已写入到文件：{csv_file_path}")

In [ ]:
from langchain.chat_models import ChatOpenAI
def post_query(query):
    model = ChatOpenAI(
            openai_api_key="sk-b3RjMZgQjoCCQzqA2dD4F232532645Ab9948062611E45171",
            openai_api_base="http://10.13.14.16:9999/v1",
            model_name="qwen2-72b",
            temperature=0.1,
        )
    answer = model.invoke(query).content
    return answer